# 시계열 머신러닝 모델링

이 노트북은 시계열 데이터에 대해 특성 엔지니어링(Feature Engineering)을 수행하고, 이를 입력으로 사용하여 머신러닝 모델(Linear Regression, RandomForest, XGBoost)을 학습시켜 미래 값을 예측하는 과정을 보여줍니다.
시계열 자체의 순서나 시간적 의존성을 직접 모델링하는 ARIMA나 LSTM과 달리, 이 접근 방식은 과거 시점의 값(lag), 이동 통계량, 날짜 정보 등을 독립적인 특성으로 만들어 일반적인 지도 학습(Supervised Learning) 회귀 문제처럼 다룹니다.

## 사용 가이드
1. **데이터 경로 설정**: `DATA_PATH` 변수를 실제 데이터 파일 경로로 변경합니다.
2. **날짜/타겟 열 이름**: `DATE_COL`과 `TARGET_COL` 변수를 데이터에 맞게 변경합니다.
3. **특성 엔지니어링 파라미터**: 시차(`LAG_PERIODS`), 이동 윈도우(`ROLLING_WINDOWS`), 날짜 특성 사용 여부(`USE_XXX_FEATURES`) 등을 필요에 따라 조정합니다.
4. **모델 선택**: 사용할 머신러닝 모델(`MODELS`)을 리스트에 지정합니다.
5. **실행**: 전체 노트북 셀을 순차적으로 실행하여 모델 결과를 확인합니다.

## 분석 흐름
1. **라이브러리 임포트 및 설정**: 필요한 라이브러리를 불러오고 기본 설정을 합니다.
2. **데이터 로드 및 탐색**: 데이터를 불러와 기본적인 정보와 구조를 파악합니다.
3. **시계열 데이터 시각화**: 원본 데이터의 추세, 패턴, 분포 등을 시각적으로 확인합니다.
4. **시계열 분해 및 정상성 검정**: 데이터의 추세, 계절성, 잔차 성분을 분해하고 정상성 여부를 검토합니다. (머신러닝 모델은 ARIMA와 달리 정상성이 필수 조건은 아니지만, 데이터 이해를 위해 수행)
5. **특성 엔지니어링**: 시계열 데이터를 머신러닝 모델의 입력으로 사용하기 위해 다양한 특성(날짜, 시차, 이동 통계량 등)을 생성합니다.
6. **데이터 분할 및 전처리**: 생성된 특성 데이터를 훈련, 검증, 테스트 세트로 분할하고, 스케일링을 적용합니다.
7. **모델 훈련 및 평가**: 선택된 머신러닝 모델들을 훈련시키고 성능을 평가합니다.
8. **모델 비교**: 각 모델의 성능 지표를 비교하여 최적 모델을 선정합니다. (필요시 특성 중요도 확인)
9. **미래 예측**: 선택된 최적 모델을 사용하여 미래 기간의 값을 예측합니다.
10. **결론 및 인사이트**: 분석 결과와 모델 성능을 요약하고 인사이트를 도출합니다.

## 1. 사용자 입력 파라미터 설정
분석 대상 데이터, 특성 생성 방식, 사용할 모델 등 주요 파라미터를 설정합니다.
사용자의 데이터와 분석 목적에 맞게 이 섹션의 값들을 수정해야 합니다.

In [ ]:
# ===== 필수 수정 파라미터 =====
# 데이터 파일 경로 (CSV 형식 권장)
DATA_PATH = "../data/raw/your_data.csv"

# 날짜 열 이름 (데이터프레임 내 날짜/시간 정보가 있는 열)
DATE_COL = "date"

# 타겟 열 이름 (예측하려는 변수가 있는 열)
TARGET_COL = "value"

# ===== 선택적 수정 파라미터 =====
# 훈련/검증/테스트 분할 비율
TRAIN_SIZE = 0.7                         # 훈련 데이터 비율
VALIDATION_SIZE = 0.15                   # 검증 데이터 비율 (나머지는 테스트 데이터)

# 특성 엔지니어링 파라미터
USE_LAG_FEATURES = True                  # 시차(lag) 특성 사용 여부
LAG_PERIODS = [1, 7, 14, 30]             # 사용할 시차 기간
USE_ROLLING_FEATURES = True              # 이동 평균/표준편차 특성 사용 여부
ROLLING_WINDOWS = [7, 14, 30]            # 이동 윈도우 크기
USE_DATE_FEATURES = True                 # 날짜 기반 특성 사용 여부

# 사용할 머신러닝 모델
MODELS = ["LR", "RF", "XGB"]            # LR: LinearRegression, RF: RandomForest, XGB: XGBoost

# 예측 기간 (일 단위)
FORECAST_HORIZON = 30

# 랜덤 시드
RANDOM_STATE = 42

# 출력 경로
OUTPUT_DIR = '../plots'
DATA_OUTPUT_DIR = '../data/processed'

# 출력 디렉토리 생성
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DATA_OUTPUT_DIR, exist_ok=True)

## 2. 필요 라이브러리 임포트
데이터 처리(pandas, numpy), 시각화(matplotlib, seaborn), 시계열 분석(statsmodels), 머신러닝(sklearn, xgboost) 등에 필요한 라이브러리를 불러옵니다.

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 시계열 라이브러리
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# 머신러닝 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

# 경고 무시
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

# 랜덤 시드 설정 (모델 재현성을 위해)
np.random.seed(RANDOM_STATE)

## 3. 데이터 로드 및 탐색
지정된 경로의 데이터를 불러와 Pandas 데이터프레임으로 변환하고, 기본적인 정보(행/열 개수, 컬럼 타입, 통계 요약)를 확인합니다.
날짜 컬럼은 datetime 형식으로 변환하여 시계열 분석 준비를 합니다.

In [ ]:
# 데이터 로드
df = pd.read_csv(DATA_PATH)
print(f"데이터 로드 완료. 형태: {df.shape}")

# 날짜 열 처리
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df.set_index(DATE_COL, inplace=True)
print(f"'{DATE_COL}' 열을 날짜 형식으로 변환하고 인덱스로 설정했습니다.")

# 데이터 정보 확인
print("\n데이터 기본 정보:")
print(df.info())
print("\n기술 통계량:")
print(df.describe())

# 처음 몇 행 확인
print("\n처음 5개 행:")
print(df.head())

## 4. 시계열 데이터 시각화
타겟 변수의 시계열 변화와 값의 분포를 시각적으로 탐색합니다.
- **선 그래프**: 시간 흐름에 따른 타겟 변수의 전반적인 추세와 패턴을 확인합니다.
- **분포 플롯 (히스토그램, 박스플롯)**: 타겟 변수 값의 분포 형태, 중심 경향성, 이상치 존재 여부 등을 파악합니다.
결측치가 있다면 선형 보간법으로 처리합니다.

In [ ]:
# 타겟 변수 시계열 시각화
plt.figure(figsize=(14, 7))
plt.plot(df.index, df[TARGET_COL], marker='.', linestyle='-', alpha=0.8)
plt.title(f'{TARGET_COL} 시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ml_시계열_원본.png')
plt.show()

# 타겟 변수 분포
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
sns.histplot(df[TARGET_COL], kde=True)
plt.title(f'{TARGET_COL} 분포')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
sns.boxplot(y=df[TARGET_COL])
plt.title(f'{TARGET_COL} 박스플롯')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ml_타겟_분포.png')
plt.show()

# 결측치 확인 및 처리
missing_values = df[TARGET_COL].isnull().sum()
print(f"\n{TARGET_COL} 결측치 개수: {missing_values}")

if missing_values > 0:
    df[TARGET_COL] = df[TARGET_COL].interpolate(method='linear')
    print(f"{TARGET_COL} 결측치를 선형 보간하여 처리했습니다.")

## 5. 시계열 분해 및 정상성 검정
시계열 데이터의 구조적 특징을 더 깊이 이해하기 위해 구성 요소를 분해하고 정상성을 검정합니다.
- **시계열 분해**: 데이터를 추세, 계절성, 잔차로 분해하여 각 성분의 영향을 시각적으로 확인합니다. (여기서는 가법 모델 사용)
- **정상성 검정 (ADF)**: 시계열의 통계적 특성이 시간에 따라 변하지 않는지(정상성) 검정합니다. 머신러닝 모델은 정상성이 필수 조건은 아니지만, 데이터 특성 파악에 도움이 됩니다.
- **ACF/PACF 플롯**: 시계열 내 자기상관 구조를 시각적으로 확인합니다. ARIMA 모델처럼 직접적으로 파라미터 결정에 사용되지는 않지만, 데이터의 의존성 패턴을 이해하는 데 유용합니다.

In [ ]:
# 시계열 분해 (가법 모델 사용)
# 참고: seasonal_decompose는 데이터 길이가 period의 2배 이상이어야 정상 작동합니다.
#      데이터가 짧거나 주기가 불분명하면 오류가 발생할 수 있습니다.
decomposition = seasonal_decompose(df[TARGET_COL], model='additive', period=7)

plt.figure(figsize=(14, 12))
plt.subplot(4, 1, 1)
plt.plot(decomposition.observed)
plt.title('원본 시계열')
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 2)
plt.plot(decomposition.trend)
plt.title('추세 성분')
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 3)
plt.plot(decomposition.seasonal)
plt.title('계절성 성분')
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 4)
plt.plot(decomposition.resid)
plt.title('잔차')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ml_시계열_분해.png')
plt.show()

# 정상성 검정 (ADF 테스트)
adf_result = adfuller(df[TARGET_COL].dropna())
print("\nADF 정상성 검정 결과:")
print(f"ADF 통계량: {adf_result[0]:.4f}")
print(f"p-값: {adf_result[1]:.4f}")

# 정상성 판단
if adf_result[1] < 0.05:
    print("결론: 시계열이 정상적일 가능성이 높습니다 (p < 0.05).")
else:
    print("결론: 시계열이 비정상적일 가능성이 높습니다 (p >= 0.05).")

# 자기상관 및 부분 자기상관 함수
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plot_acf(df[TARGET_COL].dropna(), lags=40, alpha=0.05, ax=plt.gca())
plt.title('자기상관함수(ACF)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plot_pacf(df[TARGET_COL].dropna(), lags=40, alpha=0.05, method='ywm', ax=plt.gca())
plt.title('부분 자기상관함수(PACF)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ml_ACF_PACF.png')
plt.show()

## 6. 특성 엔지니어링
시계열 데이터를 머신러닝 모델이 학습할 수 있는 형태의 특성(feature)으로 변환하는 과정입니다.
- **날짜 특성**: 연, 월, 요일, 분기 등 날짜/시간 정보에서 파생된 특성. 주기성이나 특정 시점의 효과를 모델링하는 데 사용됩니다.
- **시차(Lag) 특성**: 과거 시점의 타겟 변수 값. 자기회귀(Autoregressive) 효과를 반영합니다. (예: 어제의 값이 오늘의 값에 영향을 미치는 경우)
- **이동(Rolling) 특성**: 과거 일정 기간(윈도우) 동안의 타겟 변수 통계량(평균, 표준편차 등). 최근 데이터의 추세나 변동성을 요약하여 특성으로 사용합니다.
생성된 특성들과 타겟 변수 간의 상관관계를 확인하여 모델링에 유용할 특성을 탐색합니다.

In [ ]:
def create_date_features(df):
    """날짜 인덱스로부터 특성을 생성합니다."""
    df_features = df.copy()
    
    # 날짜 성분 추출
    df_features['dayofweek'] = df.index.dayofweek
    df_features['quarter'] = df.index.quarter
    df_features['month'] = df.index.month
    df_features['year'] = df.index.year
    df_features['dayofyear'] = df.index.dayofyear
    df_features['dayofmonth'] = df.index.day
    df_features['weekofyear'] = df.index.isocalendar().week
    
    # 주말/평일 특성
    df_features['is_weekend'] = df_features['dayofweek'].isin([5, 6]).astype(int)
    
    # 계절 특성
    df_features['season'] = (df_features['month'] % 12 + 3) // 3
    
    return df_features

def create_lag_features(df, target_col, lag_periods):
    """시차(lag) 특성을 생성합니다."""
    df_features = df.copy()
    
    # 시차 특성 생성
    for lag in lag_periods:
        df_features[f'lag_{lag}'] = df_features[target_col].shift(lag)
    
    return df_features

def create_rolling_features(df, target_col, windows):
    """이동 윈도우(rolling) 특성을 생성합니다."""
    df_features = df.copy()
    
    # 이동 평균 및 표준편차
    for window in windows:
        df_features[f'rolling_mean_{window}'] = df_features[target_col].rolling(window=window).mean()
        df_features[f'rolling_std_{window}'] = df_features[target_col].rolling(window=window).std()
    
    return df_features

# 특성 데이터프레임 생성
features_df = df.copy()

# 날짜 특성 생성
if USE_DATE_FEATURES:
    features_df = create_date_features(features_df)
    print("날짜 기반 특성이 생성되었습니다.")

# 시차 특성 생성
if USE_LAG_FEATURES:
    features_df = create_lag_features(features_df, TARGET_COL, LAG_PERIODS)
    print("시차(lag) 특성이 생성되었습니다.")

# 이동 윈도우 특성 생성
if USE_ROLLING_FEATURES:
    features_df = create_rolling_features(features_df, TARGET_COL, ROLLING_WINDOWS)
    print("이동 윈도우(rolling) 특성이 생성되었습니다.")

# 특성 및 타겟 변수 분리
X = features_df.drop(TARGET_COL, axis=1)
y = features_df[TARGET_COL]

# 결측치 처리 (특성 생성으로 인한 초기 결측치 제거)
max_lag = 0
if USE_LAG_FEATURES:
    max_lag = max(max_lag, max(LAG_PERIODS))
if USE_ROLLING_FEATURES:
    max_lag = max(max_lag, max(ROLLING_WINDOWS))

if max_lag > 0:
    X = X.iloc[max_lag:]
    y = y.iloc[max_lag:]
    print(f"특성 생성으로 인한 결측치를 처리했습니다. 처리 후 데이터 크기: {X.shape}")

# 특성 정보 출력
print("\n생성된 특성:")
for col in X.columns:
    print(f"- {col}")

# 상관관계 시각화
plt.figure(figsize=(14, 12))
# 특성이 너무 많으면 상위 20개만 선택
if X.shape[1] > 20:
    # 타겟과의 상관관계 기준으로 상위 특성 선택
    correlations = X.apply(lambda x: x.corr(y) if x.dtype.kind in 'bifc' else 0)
    top_features = correlations.abs().nlargest(20).index
    correlation_matrix = pd.concat([X[top_features], y], axis=1).corr()
else:
    correlation_matrix = pd.concat([X, y], axis=1).corr()

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(
    correlation_matrix, mask=mask, annot=True, fmt='.2f', 
    cmap='coolwarm', vmin=-1, vmax=1
)
plt.title('특성 간 상관관계')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ml_특성_상관관계.png')
plt.show()

## 7. 데이터 분할 및 전처리
생성된 특성 데이터를 모델 학습 및 평가를 위해 훈련, 검증, 테스트 세트로 분할합니다. 시계열 데이터의 순서를 유지하며 분할하는 것이 중요합니다.
또한, 머신러닝 모델의 성능 향상과 안정적인 학습을 위해 특성 스케일링(여기서는 StandardScaler)을 적용합니다.

In [ ]:
# 훈련/검증/테스트 데이터 분할 (시간 순서 유지)
train_val_split_idx = int(len(X) * TRAIN_SIZE)
val_test_split_idx = int(len(X) * (TRAIN_SIZE + VALIDATION_SIZE))

X_train = X.iloc[:train_val_split_idx]
y_train = y.iloc[:train_val_split_idx]

X_val = X.iloc[train_val_split_idx:val_test_split_idx]
y_val = y.iloc[train_val_split_idx:val_test_split_idx]

X_test = X.iloc[val_test_split_idx:]
y_test = y.iloc[val_test_split_idx:]

print(f"데이터 분할 완료:")
print(f"- 훈련 데이터: {X_train.shape}")
print(f"- 검증 데이터: {X_val.shape}")
print(f"- 테스트 데이터: {X_test.shape}")

# 데이터 전처리 - 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val) # 검증 데이터는 transform만 적용
X_test_scaled = scaler.transform(X_test) # 테스트 데이터는 transform만 적용

print(f"\n전처리 후 훈련 데이터: {X_train_scaled.shape}")
print(f"전처리 후 검증 데이터: {X_val_scaled.shape}")
print(f"전처리 후 테스트 데이터: {X_test_scaled.shape}")

## 8. 모델 훈련 및 평가
선택된 머신러닝 모델(Linear Regression, RandomForest, XGBoost)들을 준비된 훈련 데이터로 학습시키고, 테스트 데이터로 예측 성능을 평가합니다.
- **모델 정의**: 각 모델 객체를 생성합니다. (하이퍼파라미터는 기본값 또는 간단한 설정 사용)
- **훈련**: `fit()` 메서드를 사용하여 모델을 학습시킵니다.
- **예측**: 학습된 모델로 훈련 및 테스트 데이터에 대한 예측을 수행합니다 (`predict()` 사용).
- **평가**: 예측값과 실제값을 비교하여 성능 지표(RMSE, MAE, R²)를 계산합니다. 시각화를 통해 예측 결과를 직관적으로 확인합니다.
함수(`train_and_evaluate_model`)를 사용하여 반복 작업을 효율화합니다.

In [ ]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    """모델을 훈련하고 평가합니다."""
    # 모델 훈련
    model.fit(X_train, y_train)
    
    # 예측
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # 평가 지표 계산
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    print(f"\n===== {model_name} 모델 평가 =====")
    print(f"훈련 RMSE: {train_rmse:.4f}")
    print(f"테스트 RMSE: {test_rmse:.4f}")
    print(f"훈련 MAE: {train_mae:.4f}")
    print(f"테스트 MAE: {test_mae:.4f}")
    print(f"훈련 R²: {train_r2:.4f}")
    print(f"테스트 R²: {test_r2:.4f}")
    
    # 결과 시각화
    plt.figure(figsize=(14, 7))
    
    # 시계열 예측 시각화
    plt.plot(y_train.index, y_train.values, 'b-', label='훈련 실제값')
    plt.plot(y_train.index, y_train_pred, 'r--', label='훈련 예측값')
    plt.plot(y_test.index, y_test.values, 'g-', label='테스트 실제값')
    plt.plot(y_test.index, y_test_pred, 'm--', label='테스트 예측값')
    
    plt.title(f'{model_name} 시계열 예측')
    plt.xlabel('날짜')
    plt.ylabel(TARGET_COL)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/ml_{model_name}_시계열예측.png')
    plt.show()
    
    return {
        'model': model,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'y_test_pred': y_test_pred
    }

# 모델 훈련 및 평가
model_results = {}

# 선형 회귀 모델
if "LR" in MODELS:
    lr_model = LinearRegression()
    model_results['LinearRegression'] = train_and_evaluate_model(
        lr_model, X_train_scaled, y_train, X_test_scaled, y_test, 'LinearRegression'
    )

# 랜덤 포레스트 모델
if "RF" in MODELS:
    rf_model = RandomForestRegressor(
        n_estimators=100, 
        max_depth=10, 
        random_state=RANDOM_STATE
    )
    model_results['RandomForest'] = train_and_evaluate_model(
        rf_model, X_train_scaled, y_train, X_test_scaled, y_test, 'RandomForest'
    )

# XGBoost 모델
if "XGB" in MODELS:
    xgb_model = XGBRegressor(
        n_estimators=100, 
        max_depth=5, 
        learning_rate=0.1, 
        random_state=RANDOM_STATE
    )
    model_results['XGBoost'] = train_and_evaluate_model(
        xgb_model, X_train_scaled, y_train, X_test_scaled, y_test, 'XGBoost'
    )

## 9. 모델 비교
여러 모델의 평가 결과를 비교하여 어떤 모델이 해당 데이터와 문제에 가장 적합한지 판단합니다.
- **성능 비교**: 테스트 세트에 대한 평가지표(RMSE, MAE, R²)를 표와 그래프로 비교합니다.
- **최적 모델 선정**: 가장 우수한 성능을 보인 모델을 선택합니다. (여기서는 R² 기준)
- **특성 중요도 (Tree 기반 모델)**: RandomForest나 XGBoost와 같이 특성 중요도를 제공하는 모델의 경우, 어떤 특성이 예측에 중요하게 사용되었는지 확인합니다.

In [ ]:
# 모델 비교
if model_results:
    model_comparison = pd.DataFrame({
        'Model': list(model_results.keys()),
        'Train RMSE': [results['train_rmse'] for results in model_results.values()],
        'Test RMSE': [results['test_rmse'] for results in model_results.values()],
        'Train MAE': [results['train_mae'] for results in model_results.values()],
        'Test MAE': [results['test_mae'] for results in model_results.values()],
        'Train R²': [results['train_r2'] for results in model_results.values()],
        'Test R²': [results['test_r2'] for results in model_results.values()]
    })
    
    print("\n===== 모델 성능 비교 =====")
    print(model_comparison.set_index('Model'))
    
    # 모델 성능 비교 시각화
    plt.figure(figsize=(14, 10))
    
    # RMSE 비교
    plt.subplot(2, 2, 1)
    plt.bar(model_comparison['Model'], model_comparison['Test RMSE'])
    plt.title('테스트 RMSE 비교')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # MAE 비교
    plt.subplot(2, 2, 2)
    plt.bar(model_comparison['Model'], model_comparison['Test MAE'])
    plt.title('테스트 MAE 비교')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # R² 비교
    plt.subplot(2, 2, 3)
    plt.bar(model_comparison['Model'], model_comparison['Test R²'])
    plt.title('테스트 R² 비교')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # 훈련/테스트 RMSE 비교
    plt.subplot(2, 2, 4)
    width = 0.35
    x = np.arange(len(model_comparison['Model']))
    plt.bar(x - width/2, model_comparison['Train RMSE'], width, label='훈련 RMSE')
    plt.bar(x + width/2, model_comparison['Test RMSE'], width, label='테스트 RMSE')
    plt.title('훈련/테스트 RMSE 비교')
    plt.xticks(x, model_comparison['Model'], rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/ml_모델성능비교.png')
    plt.show()
    
    # 모델 성능 비교 저장
    model_comparison.to_csv(f'{DATA_OUTPUT_DIR}/시계열_모델성능비교.csv', index=False)
    print(f"모델 성능 비교 결과가 '{DATA_OUTPUT_DIR}/시계열_모델성능비교.csv'에 저장되었습니다.")
    
    # 최고 성능 모델 선택
    best_model_name = model_comparison.iloc[model_comparison['Test R²'].argmax()]['Model']
    best_model = model_results[best_model_name]['model']
    
    print(f"\n최고 성능 모델: {best_model_name}")
    print(f"테스트 R²: {model_results[best_model_name]['test_r2']:.4f}")
    
    # 특성 중요도 시각화 (해당되는 경우)
    if best_model_name in ['RandomForest', 'XGBoost'] and hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        plt.figure(figsize=(14, 10))
        sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
        plt.title(f'{best_model_name} 특성 중요도 (상위 20개)')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/ml_특성중요도.png')
        plt.show()
        
        print("\n주요 특성 (상위 5개):")
        for i, (feature, importance) in enumerate(zip(feature_importance['feature'][:5], feature_importance['importance'][:5])):
            print(f"{i+1}. {feature}: {importance:.4f}")

## 10. 미래 예측
선택된 최적 모델을 사용하여 학습 데이터 이후의 미래 기간에 대한 예측을 수행합니다.
**주의**: 이 예측은 미래 시점의 실제 특성 값(lag, rolling 등)을 알 수 없다는 한계가 있습니다. 여기서는 단순화된 방식으로 과거 데이터를 기반으로 미래 특성을 추정하여 예측을 수행하므로, 실제 적용 시에는 예측 정확도에 한계가 있을 수 있습니다. 더 정확한 예측을 위해서는 반복적 예측(iterative forecasting) 등의 기법이 필요합니다.

In [ ]:
if 'best_model' in locals():
    # 예측 기간 설정
    last_date = X.index[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=FORECAST_HORIZON, freq='D')
    
    # 미래 데이터 특성 생성
    # 참고: 아래 코드는 미래 시점의 특성을 생성하는 간단한 예시입니다.
    #      실제로는 미래의 날짜 특성 외 다른 특성(lag, rolling 등)은 알 수 없으므로,
    #      더 정확한 예측을 위해서는 예측된 값을 다시 입력으로 사용하는 반복적 예측(iterative forecasting)
    #      또는 미래 특성을 예측하는 별도 모델링이 필요할 수 있습니다.
    #      여기서는 교육적 목적을 위해 구현을 단순화했습니다.
    future_df = pd.DataFrame(index=future_dates)
    
    # 미래 날짜 특성 생성
    if USE_DATE_FEATURES:
        future_df = create_date_features(future_df)
    
    # 최근 데이터를 사용하여 시차 특성 생성 (단순화된 방식)
    if USE_LAG_FEATURES:
        # 가장 최근 데이터(y)를 사용하여 미래 시점의 lag 특성 추정
        recent_data = df[TARGET_COL].copy()
        for i, date in enumerate(future_dates):
            # 이 방식은 실제 미래 값을 모르므로 한계가 있음
            for lag in LAG_PERIODS:
                lag_idx = len(recent_data) - lag + i
                # 과거 데이터가 있는 경우에만 사용, 없으면 NaN (추후 fillna)
                future_df.loc[date, f'lag_{lag}'] = recent_data.iloc[lag_idx] if lag_idx >= 0 and lag_idx < len(recent_data) else np.nan

    # 최근 데이터를 사용하여 이동 윈도우 특성 생성 (단순화된 방식)
    if USE_ROLLING_FEATURES:
        # 이 방식도 실제 미래 값을 모르므로 한계가 있음
        # 가장 최근 rolling 값으로 채우거나, 다른 추정 방식 필요
        # 여기서는 결측치 처리 단계에서 fillna(0) 등으로 처리됨
        pass # Rolling 특성은 미래 시점 값을 예측하기 어려움

    # 컬럼 순서 맞추기 (훈련 데이터와 동일하게)
    for col in X.columns:
        if col not in future_df.columns:
            future_df[col] = np.nan # 없는 특성은 NaN으로 초기화
    
    future_df = future_df[X.columns]
    
    # 결측치 처리 (Lag, Rolling 등 미래를 알 수 없는 특성 처리)
    # 주의: fillna(0)은 간단한 처리 방식이며, 데이터 특성에 따라 ffill, 평균값 등으로 변경 고려
    future_df = future_df.fillna(0)
    
    # 전처리 적용
    future_features_scaled = scaler.transform(future_df)
    
    # 예측
    future_predictions = best_model.predict(future_features_scaled)
    
    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'date': future_dates,
        'forecast': future_predictions
    }).set_index('date')
    
    # 최근 실제 데이터와 예측 시각화
    plt.figure(figsize=(14, 7))
    
    # 과거 데이터 (최근 90일)
    history_period = min(90, len(df))
    plt.plot(df.index[-history_period:], df[TARGET_COL][-history_period:], 'b-', label='과거 실제값')
    
    # 테스트 데이터의 예측값
    plt.plot(y_test.index, model_results[best_model_name]['y_test_pred'], 'g--', label='테스트 예측값')
    
    # 미래 예측
    plt.plot(forecast_df.index, forecast_df['forecast'], 'r--', label='미래 예측값')
    
    # 테스트/예측 구분선
    plt.axvline(x=last_date, color='k', linestyle='-', alpha=0.3)
    plt.title(f'{best_model_name} 모델을 사용한 {FORECAST_HORIZON}일 예측')
    plt.xlabel('날짜')
    plt.ylabel(TARGET_COL)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/ml_미래예측.png')
    plt.show()
    
    # 예측 결과 저장
    forecast_df.to_csv(f'{DATA_OUTPUT_DIR}/시계열_예측결과.csv')
    print(f"예측 결과가 '{DATA_OUTPUT_DIR}/시계열_예측결과.csv'에 저장되었습니다.")
    
    # 예측 요약
    print("\n===== 예측 요약 =====")
    print(f"예측 기간: {forecast_df.index[0].strftime('%Y-%m-%d')} ~ {forecast_df.index[-1].strftime('%Y-%m-%d')}")
    print(f"예측 범위: {forecast_df['forecast'].min():.4f} ~ {forecast_df['forecast'].max():.4f}")
    print(f"평균 예측값: {forecast_df['forecast'].mean():.4f}")

## 11. 결론 및 인사이트
전체 분석 과정과 모델 성능 결과를 요약하고, 데이터 특성, 사용된 특성, 최적 모델 및 성능 등에 대한 인사이트를 정리합니다. 향후 모델 개선 방향이나 추가 분석 아이디어를 제시할 수 있습니다.

In [ ]:
if 'model_results' in locals() and model_results:
    print("\n" + "="*50)
    print("시계열 머신러닝 모델링 요약")
    print("="*50)
    
    print(f"\n1. 데이터 정보:")
    print(f"   - 데이터 포인트 수: {len(df)}")
    print(f"   - 사용된 특성 수: {X.shape[1]}")
    
    print(f"\n2. 모델 성능:")
    for model_name, results in model_results.items():
        print(f"   - {model_name}: 테스트 RMSE = {results['test_rmse']:.4f}, R² = {results['test_r2']:.4f}")
    
    print(f"\n3. 최고 성능 모델: {best_model_name}")
    print(f"   - 테스트 R²: {model_results[best_model_name]['test_r2']:.4f}")
    print(f"   - 테스트 RMSE: {model_results[best_model_name]['test_rmse']:.4f}")
    
    print("\n시계열 머신러닝 모델링이 완료되었습니다!") 